In [1]:
import mne 
import numpy as np
import os

In [11]:
raw_data_path = r'D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw.fif'
raw = mne.io.read_raw_fif(raw_data_path, preload=True)

Opening raw data file D:\Priming_name\Pilot02_cogwear\sub-Pilot02\ses-S001\eeg\sub-Pilot02_ses-S001_task-Default_run-001_eeg-raw.fif...
    Range : 0 ... 162659 =      0.000 ...   650.636 secs
Ready.
Reading 0 ... 162659  =      0.000 ...   650.636 secs...


In [9]:
events = mne.find_events(raw, stim_channel='Status', shortest_event=1)
print(np.unique(events[:, 2]))

ValueError: Missing channels from ch_names required by include:
['Status']

In [4]:
sfreq = raw.info['sfreq']
prim_codes = [71, 72, 73, 74]
face_codes = list(range(1, 21))

for code in prim_codes:
    prim_samples = events[events[:, 2] == code, 0]
    print(f"\nCode {code}:")
    for s in prim_samples[:5]:  # just look at first 5 trials per condition
        # find nearest face event (any 1–20)
        diffs = events[np.isin(events[:, 2], face_codes), 0] - s
        nearest_idx = np.argmin(np.abs(diffs))
        delay_ms = diffs[nearest_idx] / sfreq * 1000
        print(f"  nearest face = {events[np.isin(events[:, 2], face_codes)][nearest_idx, 2]}, delay = {delay_ms:.1f} ms")



Code 71:
  nearest face = 19, delay = 476.6 ms
  nearest face = 9, delay = 408.2 ms
  nearest face = 15, delay = 464.8 ms
  nearest face = 7, delay = 556.6 ms
  nearest face = 3, delay = 484.4 ms

Code 72:
  nearest face = 19, delay = 535.2 ms
  nearest face = 13, delay = 482.4 ms
  nearest face = 1, delay = 443.4 ms
  nearest face = 1, delay = 392.6 ms
  nearest face = 19, delay = 447.3 ms

Code 73:
  nearest face = 8, delay = 552.7 ms
  nearest face = 12, delay = 416.0 ms
  nearest face = 10, delay = 535.2 ms
  nearest face = 14, delay = 515.6 ms
  nearest face = 6, delay = 396.5 ms

Code 74:
  nearest face = 16, delay = 492.2 ms
  nearest face = 14, delay = 505.9 ms
  nearest face = 16, delay = 521.5 ms
  nearest face = 8, delay = 447.3 ms
  nearest face = 18, delay = 396.5 ms


In [5]:
new_trigger_offset = 100  # 71->171, 72->172, etc.
new_events = []

for prim_code in [71, 72, 73, 74]:
    prim_samples = events[events[:,2] == prim_code, 0]
    face_samples = events[np.isin(events[:,2], range(1,21)), 0]
    face_codes = events[np.isin(events[:,2], range(1,21)), 2]

    for s in prim_samples:
        # compute delays to all face triggers
        diffs = (face_samples - s).astype(float)  # <- convert to float
        diffs[diffs < 0] = np.inf  # ignore faces before the prime
        if np.all(diffs == np.inf):
            continue  # no future face found
        nearest_idx = np.argmin(diffs)
        face_sample = face_samples[nearest_idx]
        new_code = prim_code + new_trigger_offset
        new_events.append([face_sample, 0, new_code])


In [6]:
out_suffix = '_face_onset_copy'

In [7]:
all_events = np.vstack([events, np.array(new_events, int)])
all_events = all_events[np.argsort(all_events[:, 0])]

# save events
out_dir = os.path.dirname(raw_data_path)
base = os.path.splitext(os.path.basename(raw_data_path))[0]
out_events_fname = os.path.join(out_dir, base + out_suffix + '-events.fif')
mne.write_events(out_events_fname, all_events)
print(f"Saved new events to: {out_events_fname}")

Saved new events to: D:\Priming_name\Biosemi\2025-10-07-pilot02-block03_face_onset_copy-events.fif


C:\Users\LSA\AppData\Local\Temp\ipykernel_16908\373600613.py:8: RuntimeWarning: This filename (D:\Priming_name\Biosemi\2025-10-07-pilot02-block03_face_onset_copy-events.fif) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst or _eve.txt
  mne.write_events(out_events_fname, all_events)
